## Q1 Data analysis

import libraries to use

In [1]:
import pandas as pd
import numpy as np
import requests
import pandas_datareader.data as web
from datetime import datetime
from bs4 import BeautifulSoup
from io import StringIO
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

FERD(US Federal Reserve Economic Data) API is used with Pandas to extract data of below 3 rates:  
US Federal Rund Rate(US_FFR), EU Marginal Lending Facility Rate(EU_MLFR), UK Sterling Overnight Index Average(UK_SONIA)  
UK_LIBOR is discontinued in 2023 June so UK_SONIA is used from July.

In [2]:
# Define date range
start = datetime(2023, 5, 1)
end = datetime(2024, 4, 30)

In [3]:
# U.S. Federal Funds Rate (FRED symbol: 'FEDFUNDS')
US_FFR = web.DataReader('FEDFUNDS', 'fred', start, end)

# Europe: Marginal Lending Facility Rate (ECB) (FRED symbol: 'ECBMLFR')  
EU_MLFR = web.DataReader('ECBMLFR', 'fred', start, end)

#Daily Sterling Overnight Index Average (SONIA) Rate (FRED symbol: 'IUDSOIA')
UK_SONIA = web.DataReader('IUDSOIA', 'fred', start, end)

Data crawling from Reserve Bank of Australia website is used to extract Australia/Reserve Bank of Australia: Cash Rate Target(AU_CRT).

In [4]:
# Target URL
url = 'https://www.rba.gov.au/statistics/cash-rate/'

# Get page content
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Targeting table
table = soup.find('table')

dates = []
rates = []

# Parse each row
for row in table.find_all('tr'):
    cells = row.find_all(['th', 'td'])
    if len(cells) >= 3:
        try:
            # Extract and convert date
            date_str = cells[0].text.strip()
            date_obj = datetime.strptime(date_str, '%d %b %Y')
            
            # Extract rate
            rate_str = cells[2].text.strip()
            rate = float(rate_str)

            # Filter by date range: May 2023 to April 2024
            if datetime(2023, 5, 1) <= date_obj <= datetime(2024, 4, 30):
                dates.append(date_obj)
                rates.append(rate)
        except Exception:
            continue  # Skip if parsing fails

# Create DataFrame
AU_CRT = pd.DataFrame({'Date': dates, 'Cash Rate Target (%)': rates})
AU_CRT = AU_CRT.sort_values('Date')

print(AU_CRT.head())

        Date  Cash Rate Target (%)
9 2023-05-03                  3.85
8 2023-06-07                  4.10
7 2023-07-05                  4.10
6 2023-08-02                  4.10
5 2023-09-06                  4.10


Valet API from Bank of Canada is used to extract Canadian Overnight Repo Rate Average (CA_CORRA).

In [5]:
# URL for CSV download of CORRA (AVG.INTWO) using Valet API
url = "https://www.bankofcanada.ca/valet/observations/AVG.INTWO/csv?start_date=2023-05-01&end_date=2024-04-30"

response = requests.get(url)
csv_data = response.text
CA_CORRA = pd.read_csv(StringIO(csv_data), skiprows=8)

# Clean and rename
CA_CORRA = CA_CORRA.rename(columns={"date": "Date", "AVG.INTWO": "CORRA (%)"})
CA_CORRA["Date"] = pd.to_datetime(CA_CORRA["Date"])
CA_CORRA["CORRA (%)"] = pd.to_numeric(CA_CORRA["CORRA (%)"], errors='coerce')

CA_CORRA = pd.DataFrame(CA_CORRA)
print(CA_CORRA.head())

        Date  CORRA (%)
0 2023-05-01        4.5
1 2023-05-02        4.5
2 2023-05-03        4.5
3 2023-05-04        4.5
4 2023-05-05        4.5


ECB data Portal API is used to extract GBP 3M London Interbank Offered Rate(UK_LIBOR).

In [6]:
# URL for ECB 3-month GBP Fixed Short-Term Rate
url = "https://data-api.ecb.europa.eu/service/data/FM/M.GB.GBP.RT.MM.GBP3MFSR_.HSTA?format=jsondata"

# Perform the request
response = requests.get(url)
data = response.json()

# Navigate into structure
series = data['dataSets'][0]['series']
obs = list(series.values())[0]['observations']

# Extract time periods
time_periods = data['structure']['dimensions']['observation'][0]['values']

# Convert to DataFrame
records = [
    {"Date": time_periods[int(k)]['name'], "Rate": v[0]}
    for k, v in obs.items()
]

UK_LIBOR = pd.DataFrame(records)
UK_LIBOR['Date'] = pd.to_datetime(UK_LIBOR['Date'])
UK_LIBOR['Rate'] = pd.to_numeric(UK_LIBOR['Rate'])

print(UK_LIBOR.head())

        Date       Rate
0 1986-01-01  12.863650
1 1986-02-01  12.718760
2 1986-03-01  11.812500
3 1986-04-01  10.576709
4 1986-05-01  10.265625


Data cleaning process
1) store data into data frames
2) check data type
3) handle empty date or over flowing date
4) handle empty data
5) merge into one data frame
6) make UK_LIBOR/SONIA column(May, June 2023 = LIBOR, other = SONIA)

In [7]:
# Create date range
date_range = pd.date_range(start="2023-05-01", end="2024-04-30", freq='D')

# Create empty DataFrame with the date column
base_rate = pd.DataFrame({'DATE': date_range})

In [8]:
# Merge data frames into one, base_rate
base_rate = pd.merge(base_rate, US_FFR, on = "DATE", how = "outer")
base_rate = pd.merge(base_rate, EU_MLFR, on = "DATE", how = "outer")
base_rate = pd.merge(base_rate, UK_SONIA, on = "DATE", how = "outer")
base_rate = base_rate.rename(columns={"DATE": "Date"})
base_rate = pd.merge(base_rate, AU_CRT, on = "Date", how = "left")
base_rate = pd.merge(base_rate, CA_CORRA, on = "Date", how = "left")
base_rate = pd.merge(base_rate, UK_LIBOR, on = "Date", how = "left")
base_rate = base_rate.rename(columns={"FEDFUNDS": "US_FFR","ECBMLFR":"EU_MLFR","IUDSOIA":"UK_SONIA","Cash Rate Target (%)":"AU_CRT","CORRA (%)":"CA_CORRA","Rate":"UK_LIBOR"})
base_rate = base_rate.ffill().bfill()
base_rate

,Date,US_FFR,EU_MLFR,UK_SONIA,AU_CRT,CA_CORRA,UK_LIBOR
0,2023-05-01,5.06,3.75,4.1792,3.85,4.50,4.728291
1,2023-05-02,5.06,3.75,4.1792,3.85,4.50,4.728291
2,2023-05-03,5.06,3.75,4.1778,3.85,4.50,4.728291
3,2023-05-04,5.06,3.75,4.1779,3.85,4.50,4.728291
4,2023-05-05,5.06,3.75,4.1777,3.85,4.50,4.728291
...,...,...,...,...,...,...,...
361,2024-04-26,5.33,4.75,5.1999,4.35,5.00,5.301750
362,2024-04-27,5.33,4.75,5.1999,4.35,5.00,5.301750
363,2024-04-28,5.33,4.75,5.1999,4.35,5.00,5.301750
364,2024-04-29,5.33,4.75,5.2000,4.35,5.00,5.301750


In [9]:
#make new column UK_LIBOR/SONIA, May, June 2023 = LIBOR, SONIA afterwards
base_rate['UK_LIBOR/SONIA'] = base_rate.apply(
    lambda row: row['UK_LIBOR'] if row['Date'].month in [5, 6] and row['Date'].year == 2023
    else row['UK_SONIA'],
    axis=1
)
print(base_rate.head())

        Date  US_FFR  EU_MLFR  UK_SONIA  AU_CRT  CA_CORRA  UK_LIBOR  \
0 2023-05-01    5.06     3.75    4.1792    3.85       4.5  4.728291   
1 2023-05-02    5.06     3.75    4.1792    3.85       4.5  4.728291   
2 2023-05-03    5.06     3.75    4.1778    3.85       4.5  4.728291   
3 2023-05-04    5.06     3.75    4.1779    3.85       4.5  4.728291   
4 2023-05-05    5.06     3.75    4.1777    3.85       4.5  4.728291   

   UK_LIBOR/SONIA  
0        4.728291  
1        4.728291  
2        4.728291  
3        4.728291  
4        4.728291  


In [10]:
# Keep only the last record of each month
base_rate_month_end = base_rate.groupby([base_rate['Date'].dt.year, base_rate['Date'].dt.month]).tail(1).reset_index(drop=True)
base_rate = base_rate_month_end

Q1:
Create a line graph with all 5 rates that show the monthly Interest Rate values.

In [11]:
app = dash.Dash(__name__)

rate_columns = [c for c in base_rate.columns if c != 'Date']

app.layout = html.Div([
    html.H2("Monthly Central Bank Interest Rates (May 2023 - April 2024)"),
    dcc.Dropdown(
        id='rate-dropdown',
        options=[{'label': col, 'value': col} for col in rate_columns],
        value=rate_columns,
        multi=True
    ),
    dcc.Graph(id='rates-graph')
])

@app.callback(
    Output('rates-graph', 'figure'),
    [Input('rate-dropdown', 'value')]
)
def update_graph(selected_rates):
    fig = go.Figure()
    for col in selected_rates:
        fig.add_trace(go.Scatter(
            x=base_rate['Date'],
            y=base_rate[col],
            mode='lines+markers',
            name=col
        ))
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Interest Rate (%)",
        template='plotly_white'
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

Q2:
The Industry Standard is Base Rate (Reserve/Central Bank Interest Rate) minus 0.5%.
Create a bar graph that shows the difference between Uber’s Cash Pool Yields and the Industry
Standard Rates and order them from greatest variance (negative) to least variance (positive).
Some yields may be better than the Industry Standard rates.


In [12]:
# Select relevant columns and rename for readability
cols = {
    'US_FFR': 'USD',
    'UK_LIBOR/SONIA': 'GBP', 
    'EU_MLFR': 'EUR',
    'AU_CRT': 'AUD',
    'CA_CORRA': 'CAD'
}

# Calculate mean for each rate
mean_rates = {k: base_rate[old].mean() for old, k in cols.items()}

# Uber's yields as provided
uber_yields = {
    'USD': 4.0,
    'GBP': 3.5,
    'EUR': 4.5,
    'AUD': 3.5,
    'CAD': 4.5
}

# Calculate industry standard: mean - 0.5
industry_standard = {ccy: rate - 0.5 for ccy, rate in mean_rates.items()}

# Calculate variance: Uber yield - industry standard
variance = {ccy: uber_yields[ccy] - industry_standard[ccy] for ccy in uber_yields}

# Sort by variance
variance_sorted = dict(sorted(variance.items(), key=lambda item: item[1]))

# Plot
fig = px.bar(
    x=list(variance_sorted.values()),
    y=list(variance_sorted.keys()),
    orientation='h',
    color=list(variance_sorted.values()),
    color_continuous_scale='RdBu',
    labels={'x': 'Variance (%)', 'y': 'Currency'},
    title="Uber Yield vs Industry Standard (Base Rate - 0.5%)"
)
fig.add_vline(x=0, line_width=1, line_dash="dash", line_color="black")
fig.show()

Q3:
Create a data visualization of your choice to represent a trend analysis of the prior 12 months (May 2023 to April 2024) that provide key insights.

In [13]:
def plot_uber_vs_industry(df, currency_map, uber_yields):
    """
    df: DataFrame with all base rates, must have 'Date' column
    currency_map: dict mapping currency label (e.g. 'USD') to column name in df (e.g. 'US_FFR')
    uber_yields: dict mapping currency label to Uber yield (float)
    """

    for ccy, col in currency_map.items():
        if col not in base_rate.columns:
            print(f"Column {col} not in dataframe, skipping {ccy}.")
            continue
        
        temp = base_rate[['Date', col]].copy()
        temp.rename(columns={col: 'Base_Rate'}, inplace=True)
        temp['Industry_Standard'] = temp['Base_Rate'] - 0.5
        temp['Uber_Yield'] = uber_yields[ccy]
        
        # Prepare for grouped bar chart
        plot_df = temp.melt(id_vars='Date', value_vars=['Industry_Standard', 'Uber_Yield'],
                            var_name='Type', value_name='Rate')
        
        fig = px.bar(
            plot_df,
            x='Date',
            y='Rate',
            color='Type',
            barmode='group',
            labels={'Rate': 'Interest Rate (%)', 'Date': 'Month'},
            title=f"Uber Yield vs Industry Standard ({ccy})"
        )
        fig.show()


currency_map = {
    'USD': 'US_FFR',
    'GBP': 'UK_LIBOR/SONIA',
    'EUR': 'EU_MLFR',
    'AUD': 'AU_CRT',
    'CAD': 'CA_CORRA'
}

uber_yields = {
    'USD': 4.0,
    'GBP': 3.5,
    'EUR': 4.5,
    'AUD': 3.5,
    'CAD': 4.5
}

plot_uber_vs_industry(base_rate, currency_map, uber_yields)

In [14]:

# Select the relevant columns for the base rates
plot_cols = {
    'US_FFR': 'USD',
    'UK_LIBOR/SONIA': 'GBP', 
    'EU_MLFR': 'EUR',
    'AU_CRT': 'AUD',
    'CA_CORRA': 'CAD'
}
df_plot = base_rate[list(plot_cols.keys()) + ['Date']].rename(columns=plot_cols)

# Melt for multi-line plotting
df_melt = df_plot.melt(id_vars='Date', var_name='Currency', value_name='Base Rate')

# Plot all lines
fig = px.line(
    df_melt,
    x='Date',
    y='Base Rate',
    color='Currency',
    markers=True,
    title="Central Bank Base Rates Trend (May 2023 – April 2024)",
    labels={'Base Rate': 'Interest Rate (%)', 'Date': 'Month'}
)
fig.show()

Continued on Q1 ML